<a href="https://colab.research.google.com/github/yerfor/GeneFacePlusPlus/blob/main/inference/genefacepp_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check GPU

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Installation

In [ ]:
# install pytorch3d, about 15s
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("2.1.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [ ]:
# install dependencies, about 5~10 min
!pip install tensorboard==2.13.0 tensorboardX==2.6.1
!pip install pyspy==0.1.1
!pip install protobuf==3.20.3
!pip install scipy==1.9.1
!pip install kornia==0.5.0
!pip install trimesh==3.22.0
!pip install einops==0.6.1 torchshow==0.5.1
!pip install imageio==2.31.1 imageio-ffmpeg==0.4.8
!pip install scikit-learn==1.3.0 scikit-image==0.21.0
!pip install av==10.0.0 lpips==0.1.4
!pip install timm==0.9.2 librosa==0.9.2
!pip install openmim==0.3.9
!mim install mmcv==2.1.0 # use mim to speed up installation for mmcv
!pip install transformers==4.33.2
!pip install pretrainedmodels==0.7.4
!pip install ninja==1.11.1
!pip install faiss-cpu==1.7.4
!pip install praat-parselmouth==0.4.3 moviepy==1.0.3
!pip install mediapipe==0.10.7
!pip install --upgrade attr
!pip install beartype==0.16.4 gateloop_transformer==0.4.0
!pip install torchode==0.2.0 torchdiffeq==0.2.3
!pip install hydra-core==1.3.2 pandas==2.1.3
!pip install pytorch_lightning==2.1.2
!pip install httpx==0.23.3
!pip install gradio==4.16.0
!pip install gdown
!pip install pyloudnorm webrtcvad pyworld==0.2.1rc0 pypinyin==0.42.0
!pip install PyMCubes


In [ ]:
# RESTART kernel to make sure runtime is correct if you meet runtime errors
# import os
# os.kill(os.getpid(), 9)

Clone code and download checkpoints

In [ ]:
# clone GeneFacePlusPlus repo from github
!git clone https://github.com/yerfor/GeneFacePlusPlus
%cd GeneFacePlusPlus

In [ ]:
# replacing all 'c++14' with 'c++17' for compatible in CUDA 12
# special thanks to @yyheart in https://github.com/yerfor/GeneFacePlusPlus/issues/12
import glob
all_files = glob.glob("modules/radnerfs/*.py") + glob.glob("modules/radnerfs/*/*.py") + glob.glob("modules/radnerfs/*/*/*.py") + glob.glob("modules/radnerfs/*/*/*/*.py")
for file in all_files:
  with open(file) as f:
    lines = f.readlines()
    lines = [line.replace('-std=c++14', '-std=c++17') for line in lines]
    lines = [line.replace('\'-use_fast_math\'', '') for line in lines]
    if 'backend' in file:
      print(lines)
  with open(file, 'w') as f:
    f.writelines(lines)

# build extensions, it takes a long time (about 10 min)
!bash docs/prepare_env/install_ext.sh

In [ ]:
# download pretrained ckpts & third-party ckpts from google drive, about 1 min

%mkdir -p data/binary/videos/May
%cd data/binary/videos/May
!gdown https://drive.google.com/uc?id=16fNJz5MbOMqHYHxcK_nPP4EPBXWjugR0
%cd ../../../..

%cd deep_3drecon/BFM
!gdown https://drive.google.com/uc?id=1SPM3IHsyNAaVMwqZZGV6QVaV7I2Hly0v
!gdown https://drive.google.com/uc?id=1MSldX9UChKEb3AXLVTPzZQcsbGD4VmGF
!gdown https://drive.google.com/uc?id=180ciTvm16peWrcpl4DOekT9eUQ-lJfMU
!gdown https://drive.google.com/uc?id=1KX9MyGueFB3M-X0Ss152x_johyTXHTfU
!gdown https://drive.google.com/uc?id=19-NyZn_I0_mkF-F5GPyFMwQJ_-WecZIL
!gdown https://drive.google.com/uc?id=11ouQ7Wr2I-JKStp2Fd1afedmWeuifhof
!gdown https://drive.google.com/uc?id=18ICIvQoKX-7feYWP61RbpppzDuYTptCq
!gdown https://drive.google.com/uc?id=1VktuY46m0v_n_d4nvOupauJkK4LF6mHE
%cd ../..

%mkdir -p checkpoints
%cd checkpoints
!gdown https://drive.google.com/uc?id=1O5C1vK4yqguOhgRQ7kmYqa3-E8q5H_65
!unzip motion2video_nerf.zip
%mkdir audio2motion_vae
%cd audio2motion_vae
!gdown https://drive.google.com/uc?id=1Qg5V-1-IyEgAOxb2PbBjHpYkizuy6njf
!gdown https://drive.google.com/uc?id=1bKY5rn3vcAkv-2m1mui0qr4Fs38jEmy-
%cd ..

%cd ..


Inference sample

In [ ]:
# sample inference, about 3 min
!python inference/genefacepp_infer.py \
--a2m_ckpt=checkpoints/audio2motion_vae \
--head_ckpt= \
--torso_ckpt=checkpoints/motion2video_nerf/may_torso \
--drv_aud=data/raw/val_wavs/MacronSpeech.wav \
--out_name=may_demo.mp4 \
--low_memory_usage

Display output video

In [ ]:
# borrow code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys
import glob

mp4_name = './may_demo.mp4'

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))